### MLM1.2本地demo

# 文本->向量

### model ernie-3.0-bash-zh

In [1]:
from functools import lru_cache
from transformers import BertTokenizer, ErnieModel,ErnieForMaskedLM


/Users/alexfeng/miniconda3/envs/esv1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# #模型下载
# from modelscope import snapshot_download
# model_dir = snapshot_download('gabrielpondc/ernie-3.0-base-zh')

In [3]:
# model_dir

In [4]:
# 模型加载测试
# tokenizer = BertTokenizer.from_pretrained("/Users/alexfeng/Desktop/alex/code/work_lr/es/hgmodel/ernie-3.0-base-zh")
# model = ErnieModel.from_pretrained("/Users/alexfeng/Desktop/alex/code/work_lr/es/hgmodel/ernie-3.0-base-zh")


In [5]:
# 构建词嵌入模型
@lru_cache()
def vec_model(model_path):
    """获取模型与分词器
    注意此方法 仅适用ErnieModel类型模型
    Returns:
        model,tokenizer
    """
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = ErnieModel.from_pretrained(model_path)
    model.eval()
    return model, tokenizer

def text2vec(text, model=None, tokenizer=None, dtype='list', flatten=True):
    """文本转换为向量
    Args:
        text (_type_): _description_
        model (_type_, optional): _description_. Defaults to None.
        tokenizer (_type_, optional): _description_. Defaults to None.
        dtype (str, optional): _description_. Defaults to 'list'.
        flatten (bool, optional): _description_. Defaults to True.

    Returns:
        _type_: _description_
    """
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    vec = output[1]
    if flatten:
        vec = vec.flatten()
    if dtype == 'list':
        vec = vec.tolist()
    return vec

In [6]:
ernieModel_path ='/Users/alexfeng/Desktop/alex/code/work_lr/es/hgmodel/ernie-3.0-base-zh'
ernie_model, tokenizer = vec_model(ernieModel_path)

In [7]:
# ernie_model

In [8]:
# 文本向量测试
# 需要预测的文本
text = """
在繁华的都市中心，一栋看似普通的高级公寓内，发生了一起让人困惑的失踪案。主角杰克，是一位年轻而有才华的私人侦探，他接到了这个案件，这是他职业生涯中最具挑战性的一次。

失踪的是一位名叫艾米丽的年轻女子，她在一个晚上突然消失，没有留下任何线索。艾米丽的家人急切中带着绝望，他们找到了杰克，希望他能找到任何可能的线索。杰克开始调查这栋公寓的每个角落，询问邻居，查看监控录像，但所有的证据都似乎指向一个死胡同。

在调查过程中，杰克遇到了一位名叫汤姆的神秘男子，他自称是艾米丽的朋友，但他的话中似乎隐藏着许多未说出口的秘密。随着杰克深入挖掘，他发现这个案件似乎与一起五年前的未解案件有关。那也是一个女子失踪的案件，而且那个女子和艾米丽长得惊人的相似。

杰克逐渐发现这两个案件背后隐藏着一个深度的阴谋，涉及到了一些高层次的人物和一个秘密组织。随着调查的深入，他不仅要面对来自未知力量的威胁，还要挑战自己的道德界限。

在一个风雨交加的夜晚，杰克终于揭开了真相，那是一个关于爱恨情仇、背叛与救赎的复杂故事。原来，艾米丽的失踪只是一个更大计划的开始，而杰克不仅成功解救了艾米丽，也阻止了一个可能引发更多无辜牺牲的阴谋。

故事以杰克和艾米丽在医院的一幕结束，尽管风波尘埃落定，但他们的眼中依然充满了对未来的不确定和担忧。杰克意识到，这个城市的每一个角落都可能隐藏着未被揭露的故事，而作为一名侦探，他的旅程永无止境。
"""

text_vec = text2vec(text, ernie_model, tokenizer)
len(text_vec)

768

In [9]:
# 连接本地es
from elasticsearch import  Elasticsearch
# es = Elasticsearch()    # 默认连接本地elasticsearch
# es = Elasticsearch(['https://localhost:9200'],ca_certs=False,verify_certs=False)  # 连接本地9200端口
es = Elasticsearch('http://127.0.0.1:9200',
                #    ca_certs=False,
                   verify_certs=False
                   )
es.info()

/Users/alexfeng/miniconda3/envs/esv1/lib/python3.11/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'AlexFengdeMacBook-Pro.local',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'ujRyrbRMQRGUMtCpxwRqgw',
 'version': {'number': '7.13.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '5ca8591c6fcdb1260ce95b08a8e023559635c6f3',
  'build_date': '2021-05-19T22:22:26.081971330Z',
  'build_snapshot': False,
  'lucene_version': '8.8.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [12]:
# 创建index
index_name="multi_modal_search"
# 定义索引的映射
mapping = {
    "mappings": {
        "properties": {
            "prompt_vector": {
                "type": "dense_vector",
                "dims": 768  # 向量维度是768
            },
            "prompt_text": {
                "type": "text",
                "analyzer": "standard"  # 仍然保留文本字段，便于调试和可能的文本搜索
            },
            "url":{
                "type":"text"# 存放url
            },
            "media_type":{
                "type":"text" # 地址类型 视频/图片
            },
            "suggestions":{
                "type":"keyword" # 推荐问题
            }
        }
    }
}

# 创建索引
if es.indices.exists(index_name):
    print("exit delete")
    es.indices.delete(index_name)
response = es.indices.create(index=index_name, body=mapping)
print(response)


/Users/alexfeng/miniconda3/envs/esv1/lib/python3.11/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


exit delete
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'multi_modal_search'}


In [14]:
# 单挑插入假数据
# text="这只是一个测试"
text="这是另一个玩法"
# 构建插入document
doc={
    "prompt_text":text,
    "prompt_vector":text2vec(text=text,model=ernie_model,tokenizer=tokenizer),
    "url":"https://chatgpt.com",
    "media_type":"video",
    "suggestions":["s1","s2",
                   "s3"]
}

# 插入单条数据
response = es.index(index=index_name, body=doc)


print("插入结果:", response)

插入结果: {'_index': 'multi_modal_search', '_type': '_doc', '_id': '3hdx4pABycStGN31JCDs', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}


In [10]:

test

'\n[a,\nd,\nc]\n'

['\n', '[', 'a', ',', '\n', 'd', ',', '\n', 'c', ']', '\n']

In [ ]:
ernie_model

In [15]:
# 构建查询
query_vector=text2vec("""这只是一个测试""",
                        model=ernie_model,
                        tokenizer=tokenizer)
query = {
    "query": {
        "script_score": {
            "query": {"match_all": {}},  # 可以根据需要替换为更具体的查询
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'prompt_vector') + 1.0",
                "params": {
                    "query_vector": query_vector
                }
            }
        }
    }
}
response=es.search(index=index_name,body=query)

In [17]:
response["hits"]["hits"][0]["_source"]["suggestions"]

['s1', 's2', 's3']

In [27]:
response["hits"]["hits"][0]["_score"],response["hits"]["hits"][1]["_score"]

(2.0, 1.972322)

In [23]:
response["hits"]["hits"][0]["_source"]["prompt_text"]

'这只是一个测试'

In [19]:
a=response["hits"]["hits"][1]["_source"]["suggestions"]

In [22]:
a[0]

's1'

In [28]:
response

{'took': 10,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 2.0,
  'hits': [{'_index': 'multi_modal_search',
    '_type': '_doc',
    '_id': '2xdA4pABycStGN31iyBq',
    '_score': 2.0,
    '_source': {'prompt_text': '这只是一个测试',
     'prompt_vector': [-0.8344396352767944,
      -0.9955947399139404,
      -0.3272249698638916,
      0.5848947763442993,
      0.9709288477897644,
      -0.9999985098838806,
      -0.9998003840446472,
      0.9945009350776672,
      -0.9641563296318054,
      0.9891980886459351,
      -0.8500805497169495,
      -0.8475321531295776,
      -0.9978423118591309,
      -0.24758104979991913,
      -0.9850443601608276,
      -0.43386566638946533,
      0.8463843464851379,
      0.9988483190536499,
      -0.9882309436798096,
      0.9820237755775452,
      0.141736701130867,
      0.7620989084243774,
      0.8480131030082703,
      0.335797518491745,
      0.93